In [1]:
import pandas as pd
import numpy as np

In [2]:
# energygenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
#                         usecols=(lambda s: s.startswith('utc') | 
#                                  s.startswith('DE_solar') |
#                                  s. startswith('DE_wind')),
#                         parse_dates=[0], index_col=0)

windgenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s. startswith('DE_wind_generation')),
                        parse_dates=[0], index_col=0)

solargenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s.startswith('DE_solar')),
                        parse_dates=[0], index_col=0)

In [3]:
weatherdata = pd.read_csv("data/weather_data.csv",
                usecols=(lambda s: s.startswith('utc') | 
                         s.startswith('DE')),
                parse_dates=[0], index_col=0)

In [3]:
windspeed = pd.read_csv("data/weather_data.csv",
                usecols=(lambda s: s.startswith('utc') | 
                         s.startswith('DE_windspeed')),
                parse_dates=[0], index_col=0)

In [4]:
windspeed

,DE_windspeed_10m
utc_timestamp,
1980-01-01 00:00:00,5.7291
1980-01-01 01:00:00,5.7234
1980-01-01 02:00:00,5.7342
1980-01-01 03:00:00,5.7345
1980-01-01 04:00:00,5.6416
1980-01-01 05:00:00,5.6070
1980-01-01 06:00:00,5.5772
1980-01-01 07:00:00,5.3745
1980-01-01 08:00:00,5.4239


In [15]:
# start_remove = pd.to_datetime('2010-01-01')
start_remove = pd.to_datetime('2016-01-01')
end_remove = pd.to_datetime('2017-01-01')
windspeedformatted = windspeed.loc[(windspeed.index >= start_remove)]
windgenerationdataformatted = windgenerationdata.loc[(windgenerationdata.index >= start_remove) & (windgenerationdata.index < end_remove)]

In [16]:
windspeedformatted

,DE_windspeed_10m
utc_timestamp,
2016-01-01 00:00:00,3.3849
2016-01-01 01:00:00,3.3636
2016-01-01 02:00:00,3.3268
2016-01-01 03:00:00,3.2796
2016-01-01 04:00:00,3.1705
2016-01-01 05:00:00,3.0251
2016-01-01 06:00:00,2.8701
2016-01-01 07:00:00,2.6828
2016-01-01 08:00:00,2.6454


In [17]:
windgenerationdataformatted

,DE_wind_generation_actual
utc_timestamp,
2016-01-01 00:00:00,8579.0
2016-01-01 01:00:00,8542.0
2016-01-01 02:00:00,8443.0
2016-01-01 03:00:00,8295.0
2016-01-01 04:00:00,7319.0
2016-01-01 05:00:00,6574.0
2016-01-01 06:00:00,5624.0
2016-01-01 07:00:00,4644.0
2016-01-01 08:00:00,3733.0


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [19]:
xtr, xts, ytr, yts = train_test_split(windspeedformatted,windgenerationdataformatted, test_size = 0.33)

In [26]:
model = LinearRegression().fit(xtr,ytr)
predicted = model.predict(xts)

array([[ 4178.17744476],
       [12016.77575783],
       [ 3949.54130573],
       ...,
       [22008.60106347],
       [ 7577.8974251 ],
       [ 1122.47674814]])

In [18]:
from sklearn.model_selection import cross_val_score

model = LinearRegression()

scores_wind = cross_val_score(model, windspeedformatted, windgenerationdataformatted, cv=5)
print(scores_wind, "\naverage =", np.mean(scores_wind))

[0.81568392 0.58715522 0.45299934 0.61062025 0.65504413] 
average = 0.624300570072587


In [ ]:
'''
Todo:
take in weather data and generation data
separate data into two sets:
Solar
X = [sun related weather columns] 
Y = wind_generation_actual
Wind:
X = [wind related weather columns] 
Y = sun_generation_actual

Trim data based on timestamp
Wind generation begins at 2010-01-01
Solar generation begins at 2012-01-01

To try: if there are timestamps do I have to format data?
'''